# Preparing and uploading data to Fair Data Point (FDP) compliant with the Health-RI core shapes using SeMPyRO

**Prerequirements:** To execute this notebook in full one needs to have a running FAIR Data Point (FDP) instance with an active write access account.

I used [sample](https://github.com/Health-RI/health-ri-metadata) data as a reference for the Health-RI core shapes. The shapes are available at the repo: [shapes](https://github.com/Health-RI/health-ri-metadata/tree/develop/Formalisation(shacl)/Core/PiecesShape). And the FDP instance I used is https://fdp-test.health-ri.nl. The sample data is also available in metadata repository

FDP requires each dataset to be a part of a catalogue, therefore we need to create a catalogue. Let's see what we need to provide for that:

In [1]:
from sempyro.hri_dcat import HRICatalog

catalog_fields = HRICatalog.annotate_model()
print(catalog_fields.mandatory_fields())

['description', 'publisher', 'title']


/Users/hcvanderwerf/Library/Application Support/hatch/env/virtual/sempyro/Q415hUCa/docs/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:355: UserWarning: [typing.Annotated[pydantic_core._pydantic_core.Url, UrlConstraints(max_length=None, allowed_schemes=['http', 'https'], host_required=None, default_host=None, default_port=None, default_path=None)]] is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


Let's create a minimum catalogue with an example title and description. We also need a URI to use as a graph subject at serialization. A FDP requirement is a link pointing to a parent object, in the case of a catalogue it is FDP itself and it should be a property `is_part_of` in the range `DCTERMS.isPartOf`. More about this in the other example Usage_example_FDP.ipynb. We used the reusable code from this example.  Let's use `example.com` domain for now for this purpose:

In [4]:
from sempyro import LiteralField
from sempyro.foaf import Agent
from sempyro.hri_dcat import HRICatalog
from rdflib import URIRef, DCTERMS
from pydantic import AnyHttpUrl, Field

EX = "https://example.com"
fdp_base=input("Enter base link to FDP: ").rstrip("/")

class FDPCatalog(HRICatalog):
    is_part_of: [AnyHttpUrl] = Field(description="Link to parent object",
                                   rdf_term=DCTERMS.isPartOf,
                                   rdf_type="uri"
                                  )

# Create a class instance with the same data
fdp_catalog = FDPCatalog(
    title=[LiteralField(value="Inflammatory Bowel Disease catalogue", language="en")],
    description=[LiteralField(value="This catalogue describes the core metadata of AUMC Inflammatory Bowel Disease datasets", language="en")],
    publisher=[Agent(
                                 name=[
                                     LiteralField(value="Academic Medical Center")],
                                 identifier="https://ror.org/05wg1m734")],
    is_part_of=[fdp_base],
    dataset=[]
                        )

fdp_catalog_record = fdp_catalog.to_graph(URIRef(f"{EX}/test_catalog_1"))
print(fdp_catalog_record.serialize())




Enter base link to FDP:  https://fdp-test.healthdata.nl


@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

<https://example.com/test_catalog_1> a dcat:Catalog ;
    dcterms:description "This catalogue describes the core metadata of AUMC Inflammatory Bowel Disease datasets"@en ;
    dcterms:isPartOf <https://fdp-test.healthdata.nl> ;
    dcterms:publisher [ a foaf:Agent ;
            dcterms:identifier "https://ror.org/05wg1m734" ;
            foaf:name "Academic Medical Center" ] ;
    dcterms:title "Inflammatory Bowel Disease catalogue"@en .




Publishing a record in FDP consists of two steps: creating a record and publishing. These two actions are performed as API calls with different content types, so we need to implement methods for changing content type, creating a record and publishing the record. After that, the client looks like this:

In [5]:
username=input("Enter username: ")

In [6]:
from getpass import getpass
password = getpass()

Now we can create a client instance and publish the catalogue record to FDP:

In [7]:
from fdpAPIconnector.fdpclient import FDPClient

fdp_client = FDPClient(baseurl=fdp_base, email=username, password=password)

catalog_fdp_id = fdp_client.create(type="catalog", data=fdp_catalog_record)
fdp_client.update(
    type="catalog", subtype="meta/state", id=catalog_fdp_id, data='{"current": "PUBLISHED"}', format="json-ld"
)
print(catalog_fdp_id)

ce1f248d-3a72-41c5-a7a0-a3cd0a7d2eac


Now we need to add datasets to the catalogue. Data for example datasets will be used from [sample](https://github.com/Health-RI/health-ri-metadata) 

In [8]:
from sempyro.hri_dcat import HRIDataset
from sempyro.vcard import VCard
import dateutil.parser as parser

hri_dataset = HRIDataset(
    contact_point=[
        VCard(hasEmail=[URIRef("mailto:data-access-committee@xumc.nl")],
              full_name=["Data Access Committee of the x UMC"], hasUID="https://ror.org/05wg1m734")
    ],
    creator=[Agent(name=["Academic Medical Center"], identifier="https://ror.org/05wg1m734")],
    description=[LiteralField(value=
                              "The primary aim of the PRISMA study was to investigate the potential value of risk-tailored versus "
                              "traditional breast cancer screening protocols in the Netherlands. Data collection took place between "
                              "2014-2019, resulting in ∼67,000 mammograms, ∼38,000 surveys, ∼10,000 blood samples and ∼600 saliva "
                              "samples.")],
    issued=parser.isoparse("2024-07-01T11:11:11"),
    identifier=f"{EX}/dataset/ZLOYOJ",
    modified=parser.isoparse("2024-06-04T13:36:10.246Z"),
    publisher=[
        Agent(name=["Radboud University Medical Center"], identifier="https://ror.org/05wg1m734")],
    theme=[URIRef("http://publications.europa.eu/resource/authority/data-theme/HEAL")],
    title=[LiteralField(value="Questionnaire data of the Personalised RISk-based MAmmascreening Study (PRISMA)")],
    license=URIRef("https://creativecommons.org/licenses/by-sa/4.0/"),
    distribution=[]
)

Make sure the dataset is correctly serialized link it to the catalogue and publish it:

In [10]:
fdp_dataset_record = hri_dataset.to_graph(subject=URIRef(hri_dataset.identifier))
fdp_dataset_record.add((URIRef(hri_dataset.identifier), DCTERMS.isPartOf, URIRef(f"{fdp_base}/catalog/{catalog_fdp_id}")))
dataset_fdp_id = fdp_client.create(type="dataset", data=fdp_dataset_record)
fdp_client.update(
        type="dataset", subtype="meta/state", id=dataset_fdp_id, data='{"current": "PUBLISHED"}', format="json-ld"
    )

print(dataset_fdp_id)

DEBUG:fdpAPIconnector.fdpclient:Create metadata on https://fdp-test.healthdata.nl/dataset with the content: 
[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fdp-test.healthdata.nl:443
DEBUG:urllib3.connectionpool:https://fdp-test.healthdata.nl:443 "POST /dataset HTTP/11" 201 None
DEBUG:fdpAPIconnector.fdpclient:Update metadata on https://fdp-test.healthdata.nl/dataset/ad4638e8-94f1-4a11-8040-739b89ff41e7/meta/state with the content: 
{"current": "PUBLISHED"}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fdp-test.healthdata.nl:443
DEBUG:urllib3.connectionpool:https://fdp-test.healthdata.nl:443 "PUT /dataset/ad4638e8-94f1-4a11-8040-739b89ff41e7/meta/state HTTP/11" 200 None


ad4638e8-94f1-4a11-8040-739b89ff41e7


Now we can check the catalogue and the dataset in FDP. The next step is adding a distribution. Let's create a distribution for the dataset:

In [11]:
from sempyro.hri_dcat import HRIDistribution

hri_distribution = HRIDistribution(
    title=[LiteralField(value="CSV-distribution of the questionnaire data of the Personalised RISk-based MAmmascreening Study (PRISMA)")],
    description=[LiteralField(value="CSV file containing the questionnaire data of the PRISMA study")],
    access_url=[URIRef("https://example.com/dataset/PRISMA/questionnaire.csv")],
    media_type=URIRef("https://www.iana.org/assignments/media-types/text/csv")
)

The identifier of the distribution should be unique in the context of the dataset. Access URL is mandatory. Let's add the distribution to the dataset and publish it:

In [12]:
access_url_str = str(hri_distribution.access_url[0])
distribution_uri = URIRef(f"{EX}/distribution/{access_url_str.split('/')[-1]}")
fdp_distribution_record = hri_distribution.to_graph(subject=distribution_uri)
fdp_distribution_record.add((distribution_uri, DCTERMS.isPartOf, URIRef(f"{fdp_base}/dataset/{dataset_fdp_id}")))
distribution_fdp_id = fdp_client.create(type="distribution", data=fdp_distribution_record)
fdp_client.update(
        type="distribution", subtype="meta/state", id=distribution_fdp_id, data='{"current": "PUBLISHED"}', format="json-ld"
    )
print(distribution_fdp_id)

DEBUG:fdpAPIconnector.fdpclient:Create metadata on https://fdp-test.healthdata.nl/distribution with the content: 
[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fdp-test.healthdata.nl:443
DEBUG:urllib3.connectionpool:https://fdp-test.healthdata.nl:443 "POST /distribution HTTP/11" 201 None
DEBUG:fdpAPIconnector.fdpclient:Update metadata on https://fdp-test.healthdata.nl/distribution/ee1c426b-02b9-4892-9bab-6665513e02fe/meta/state with the content: 
{"current": "PUBLISHED"}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fdp-test.healthdata.nl:443
DEBUG:urllib3.connectionpool:https://fdp-test.healthdata.nl:443 "PUT /distribution/ee1c426b-02b9-4892-9bab-6665513e02fe/meta/state HTTP/11" 200 None


ee1c426b-02b9-4892-9bab-6665513e02fe


Now we can check the catalogue, dataset and distribution in FDP.